In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install imblearn

In [3]:
from imblearn.under_sampling import RandomUnderSampler

/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  depreca

In [4]:
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.metrics import classification_report_imbalanced

In [5]:
df_ml_data = pd.read_csv("df_with_lat_noNull.csv")

In [6]:
# df_ml_data.loc[df_ml_data["Severity"] == 4]
df_ml_data.head()

,Severity,Start_Lat,Start_Lng,Start_Time,Temperature(F),Visibility(mi),Wind_Speed(mph),Weather_Category,Pressure(in)
0,2,39.063148,-84.032608,2016-02-08T06:49:27.000Z,36.0,10.0,3.5,Cloudy,29.67
1,3,39.747753,-84.205582,2016-02-08T07:23:34.000Z,35.1,9.0,4.6,Cloudy,29.64
2,2,39.627781,-84.188354,2016-02-08T07:39:07.000Z,36.0,6.0,3.5,Cloudy,29.65
3,3,40.100590,-82.925194,2016-02-08T07:44:26.000Z,37.9,7.0,3.5,Clear,29.63
4,2,39.758274,-84.230507,2016-02-08T07:59:35.000Z,34.0,7.0,3.5,Cloudy,29.66


In [7]:
# Convert Start_Time and End_Time to datetypes
df_ml_data['Start_Time'] = pd.to_datetime(df_ml_data['Start_Time'], errors='coerce')
# df_full_data['End_Time'] = pd.to_datetime(df_full_data['End_Time'], errors='coerce')

# Extract year, month, day, hour and weekday
df_ml_data['Year']=df_ml_data['Start_Time'].dt.year
df_ml_data['Month']=df_ml_data['Start_Time'].dt.strftime('%b')
df_ml_data['Day']=df_ml_data['Start_Time'].dt.day
df_ml_data['Hour']=df_ml_data['Start_Time'].dt.hour
df_ml_data['Weekday']=df_ml_data['Start_Time'].dt.strftime('%a')

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.svm import SVC

In [9]:
import joblib
import pickle

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
class PandasDummies(BaseEstimator, TransformerMixin):
    def transform(self, X, *_):
        return pd.get_dummies(X)
    
    def fit(self, *_):
        return self

In [11]:
X = df_ml_data[["Temperature(F)", "Visibility(mi)", "Wind_Speed(mph)", "Weather_Category"]]
# X = df_panda_clean[["Temperature(F)", "Visibility(mi)", "Wind_Speed(mph)", "Weather_Category", "Pressure(in)"]]
# y = df_noweather_cond["Severity"].values.reshape(-1, 1)
X = pd.get_dummies(X)
y = df_ml_data["Severity"]
print(X.shape, y.shape)

(1785064, 9) (1785064,)


In [21]:
X

,Temperature(F),Visibility(mi),Wind_Speed(mph),Weather_Category_Clear,Weather_Category_Cloudy,Weather_Category_Low Visibility,Weather_Category_Rain,Weather_Category_Slippery,Weather_Category_UNKNOWN
0,36.0,10.0,3.5,0,1,0,0,0,0
1,35.1,9.0,4.6,0,1,0,0,0,0
2,36.0,6.0,3.5,0,1,0,0,0,0
3,37.9,7.0,3.5,1,0,0,0,0,0
4,34.0,7.0,3.5,0,1,0,0,0,0
5,34.0,7.0,3.5,0,1,0,0,0,0
6,33.3,5.0,1.2,0,1,0,0,0,0
7,37.4,3.0,4.6,1,0,0,0,0,0
8,35.6,5.0,5.8,0,0,0,1,0,0
9,37.4,3.0,4.6,1,0,0,0,0,0


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify =y)
data = X.copy()

In [14]:
# model1 = make_pipeline_imb(RandomUnderSampler(), StandardScaler(), RandomForestClassifier())
# paramgrid = {"randomforestclassifier__n_estimators": [1,5,10],
#              "randomforestclassifier__max_depth":[10,15,20] }
# clf_grid_model1 = GridSearchCV(model1, paramgrid)
# clf_grid_model1.fit(X_train, y_train)

/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and wi

/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and wi

KeyboardInterrupt: 

In [15]:
model1 = make_pipeline_imb(RandomUnderSampler(), StandardScaler(), RandomForestClassifier(n_estimators=10, max_depth=20))

model1.fit(X_train, y_train)

/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Pipeline(memory=None,
         steps=[('randomundersampler',
                 RandomUnderSampler(random_state=None, ratio=None,
                                    replacement=False, return_indices=False,
                                    sampling_strategy='auto')),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=20, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_le

In [16]:
y_pred = model1.predict(X_test)

In [18]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          2       0.67      0.37      0.65      0.48      0.49      0.24    290590
          3       0.33      0.36      0.66      0.35      0.49      0.23    142014
          4       0.05      0.42      0.72      0.08      0.55      0.30     13662

avg / total       0.54      0.37      0.65      0.42      0.49      0.24    446266



In [17]:
print(f"Training Data Score with Random Forest Classifier: {model1.score(X_train, y_train)}")
print(f"Testing Data Score with Random Forest Classifier: {model1.score(X_test, y_test)}")

Training Data Score with Random Forest Classifier: 0.3827709632072949
Testing Data Score with Random Forest Classifier: 0.37176930350956605


In [20]:
filename = 'acc_sev_prediction.pickle'
joblib.dump(model1, filename)

['acc_sev_prediction.pickle']

In [12]:
pipe = make_pipeline_imb(RandomUnderSampler(),
                         StandardScaler(),
                         RandomForestClassifier(n_estimators=10, max_depth=10))

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [13]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          2       0.67      0.34      0.69      0.46      0.49      0.23    363881
          3       0.34      0.38      0.65      0.36      0.50      0.24    178896
          4       0.05      0.43      0.70      0.08      0.55      0.30     18000

avg / total       0.54      0.36      0.67      0.41      0.49      0.23    560777



In [14]:
print(f"Training Data Score with Random Forest Classifier: {pipe.score(X_train, y_train)}")
print(f"Testing Data Score with Random Forest Classifier: {pipe.score(X_test, y_test)}")

Training Data Score with Random Forest Classifier: 0.36066743108223054
Testing Data Score with Random Forest Classifier: 0.3590108010849232


In [15]:
lg_pipe = make_pipeline_imb(RandomUnderSampler(),
                         StandardScaler(),
                         LogisticRegression())

lg_pipe.fit(X_train, y_train)
y_pred = lg_pipe.predict(X_test)

/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [16]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          2       0.66      0.46      0.56      0.54      0.51      0.25    363881
          3       0.33      0.19      0.83      0.24      0.39      0.14    178896
          4       0.04      0.47      0.64      0.08      0.55      0.30     18000

avg / total       0.53      0.37      0.64      0.43      0.47      0.22    560777



In [17]:
print(f"Training Data Score with Logistic Classifier: {lg_pipe.score(X_train, y_train)}")
print(f"Testing Data Score with Logistic Classifier: {lg_pipe.score(X_test, y_test)}")

Training Data Score with Logistic Classifier: 0.37326839961933767
Testing Data Score with Logistic Classifier: 0.37383665877880157


In [28]:
filename = 'acc_sev_prediction.pickle'
joblib.dump(pipe, filename)

['acc_sev_prediction.pickle']

In [30]:
my_model_lg = joblib.load("acc_sev_prediction.pickle")

my_model_lg.steps[-1][1].predict([[80.9, 30, 5, 0, 1, 0, 0, 0, 0]])

array([3])

In [46]:
my_model_lg.steps[-1][1].predict([[80.9, 0, 5, 0, 1, 0, 0, 0, 0]])

array([4])

In [62]:
my_model_lg.steps[-1][1].predict([[30, 20, 50, 0, 0, 1, 0, 0, 0]])

array([3])

In [48]:
my_model_lg.steps[-1][1].predict([[30, 0, 5, 0, 0, 0, 1, 0, 0]])

array([2])

In [63]:
my_model_lg.steps[-1][1].predict([[30, 0, 5, 0, 0, 0, 1, 0, 0]])

array([2])

In [27]:
grid_pipe = make_pipeline_imb(RandomUnderSampler(),
                         StandardScaler(),
                         RandomForestClassifier())
paramgrid = {"randomforestclassifier__n_estimators": [10, 30, 50],
             "randomforestclassifier__max_depth":[10, 50, 100] }
clf_grid_pipe1 = GridSearchCV(grid_pipe, paramgrid)
clf_grid_pipe1.fit(X_train, y_train)

/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and wi

/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/bhavinivyas/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and wi

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('randomundersampler',
                                        RandomUnderSampler(random_state=None,
                                                           ratio=None,
                                                           replacement=False,
                                                           return_indices=False,
                                                           sampling_strategy='auto')),
                                       ('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(bootstrap=True,
                                               

In [ ]:
print(f"Training Data Score with Random Forest Classifier grid search: {clf_grid_pipe1.score(X_train, y_train)}")
print(f"Testing Data Score with Random Forest Classifier grid search: {clf_grid_pipe1.score(X_test, y_test)}")

In [ ]:
clf_grid_pipe1.best_params_

In [ ]:
lg_grid_pipe = make_pipeline_imb(RandomUnderSampler(),
                         StandardScaler(),
                         LogisticRegression())
lg_paramgrid = {'logisticregression__C': [1,5],
#              'logisticregression__max_iter': [110,120,130,140],
             'logisticregression__solver': ['saga'],
             'logisticregression__penalty': ['l1']
            }
lg_grid_model1 = GridSearchCV(lg_grid_pipe, lg_paramgrid)
lg_grid_model1.fit(X_train, y_train)

In [ ]:
print(f"Training Data Score with Logistic regression grid search: {lg_grid_model1.score(X_train, y_train)}")
print(f"Testing Data Score with Logistic regression grid search: {lg_grid_model1.score(X_test, y_test)}")

In [ ]:
lg_grid_model1.best_params_

In [ ]:
# svm_pipe = make_pipeline_imb(RandomUnderSampler(),
#                          StandardScaler(),
#                          SVC())

# # svm_grid_model1 = GridSearchCV(svm_grid_pipe, svm_paramgrid)
# svm_pipe.fit(X_train, y_train)

In [ ]:
# print(f"Training Data Score with SVC: {svm_pipe.score(X_train, y_train)}")
# print(f"Testing Data Score with SVC: {svm_pipe.score(X_test, y_test)}")

In [ ]:
# svm_grid_pipe = make_pipeline_imb(RandomUnderSampler(),
#                          StandardScaler(),
#                          SVC(kernel='linear'))
# svm_paramgrid = {
#                 'svc__C': [0.1,1,10,50,100],
#                 "svc__gamma" : [0.0001, 0.001, 0.01, 0.1]
#             }
# svm_grid_model1 = GridSearchCV(svm_grid_pipe, svm_paramgrid)
# svm_grid_model1.fit(X_train, y_train)

In [ ]:
model = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=10, max_depth=10))
model.fit(X_train, y_train)

In [ ]:
print(f"Training Data Score with Random Forest Classifier: {model.score(X_train, y_train)}")
print(f"Testing Data Score with Random Forest Classifier: {model.score(X_test, y_test)}")

In [ ]:
rf = model.steps[-1][1]
rf.feature_importances_

In [ ]:
rf_clf_prediction = model.predict(X_test)

In [ ]:
print(classification_report(y_test, rf_clf_prediction))

In [ ]:
filename = 'acc_sev_pred.pickle'
joblib.dump(model, filename)

In [ ]:
my_model = joblib.load("acc_sev_pred.pickle")

my_model.steps[-1][1].predict([[36.9, 0.2, 25, 0, 1, 0, 0, 0, 0]])

In [ ]:
my_model.steps[-1][1].predict([[120.9, 0.2, 25, 0, 1, 1, 0, 0, 1]])

In [ ]:
my_model.steps[-1][1].predict([[36.9, 00.0, 25, 1, 1, 1, 1, 1, 1]])

In [ ]:
my_model.steps[-1][1].predict([[36.9, 0.2, 25, 0, 1, 0, 0, 0, 0]])

In [ ]:
my_model.steps[-1][1].predict([[84.9, 0.0, 25, 0, 1, 0, 0, 0, 0]])

In [ ]:
my_model.steps[-1][1].predict([[36.9, 0.0, 20.1, 1, 1, 0, 0, 0, 0]])

In [31]:
pd.get_dummies(df_ml_data)

,Severity,Temperature(F),Visibility(mi),Wind_Speed(mph),Pressure(in),Weather_Category_Clear,Weather_Category_Cloudy,Weather_Category_Low Visibility,Weather_Category_Rain,Weather_Category_Slippery,Weather_Category_UNKNOWN
0,3,36.9,10.0,0.0,29.68,1,0,0,0,0,0
1,2,37.9,10.0,0.0,29.65,1,0,0,0,0,0
2,2,36.0,10.0,3.5,29.67,0,1,0,0,0,0
3,3,35.1,9.0,4.6,29.64,0,1,0,0,0,0
4,2,36.0,6.0,3.5,29.65,0,1,0,0,0,0
5,3,37.9,7.0,3.5,29.63,1,0,0,0,0,0
6,2,34.0,7.0,3.5,29.66,0,1,0,0,0,0
7,3,34.0,7.0,3.5,29.66,0,1,0,0,0,0
8,2,33.3,5.0,1.2,29.67,0,1,0,0,0,0
9,3,37.4,3.0,4.6,29.62,1,0,0,0,0,0


In [ ]:
# lg_model = make_pipeline(PandasDummies(), StandardScaler(), LogisticRegression(solver='saga', max_iter=1))
lg_model = make_pipeline(StandardScaler(), LogisticRegression(C=0.1))
lg_model.fit(X_train, y_train)

In [ ]:
print(f"Training Data Score with Logistic Classifier: {lg_model.score(X_train, y_train)}")
print(f"Testing Data Score with Logistic Classifier: {lg_model.score(X_test, y_test)}")

In [ ]:
lg_clf_prediction = lg_model.predict(X_test)
print(classification_report(y_test, lg_clf_prediction))

In [ ]:
y_train.value_counts()

In [ ]:
# knn_model = make_pipeline(PandasDummies(), StandardScaler(), KNeighborsClassifier())
# knn_model.fit(X_train, y_train)

In [ ]:
# print(f"Training Data Score with K-Nearest Neighbor: {knn_model.score(X_train, y_train)}")
# print(f"Testing Data Score with K-Nearest Neighbor: {knn_model.score(X_test, y_test)}")

In [ ]:
df_with_latlng = pd.read_csv("df_data_with_lat.csv")

In [ ]:
df_with_latlng.head()

In [ ]:
# included latitude and longitude
X = df_with_latlng[["Start_Lat", "Start_Lng", "Temperature(F)", "Visibility(mi)", "Wind_Speed(mph)", "Weather_Category"]]
y = df_with_latlng["Severity"]
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify =y)
data = X.copy()

model_lat = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=10, max_depth=10))
model_lat.fit(X_train, y_train)

print(f"Training Data Score with Random Forest Classifier: {model_lat.score(X_train, y_train)}")
print(f"Testing Data Score with Random Forest Classifier: {model_lat.score(X_test, y_test)}")

In [ ]:
# Included Latitude, longitude, distance and TMC
X = df_with_latlng[["TMC", "City", "Start_Lat", "Start_Lng", "Temperature(F)", "Visibility(mi)", "Wind_Speed(mph)", "Weather_Category"]]
y = df_with_latlng["Severity"]
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify =y)
data = X.copy()

model_lat1 = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=10, max_depth=10))
model_lat1.fit(X_train, y_train)

print(f"Training Data Score with Random Forest Classifier: {model_lat1.score(X_train, y_train)}")
print(f"Testing Data Score with Random Forest Classifier: {model_lat1.score(X_test, y_test)}")